In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

In [2]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
houseprice_df = pd.read_sql_query('select * from houseprices',con=engine)

engine.dispose()

houseprice_df.head(10)

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,None,IR1,Lvl,AllPub,...,0,None,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,None,IR1,Lvl,AllPub,...,0,None,None,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,1,2008,WD,Normal,118000


In [5]:
import statsmodels.api as sm

Y = houseprice_df['saleprice']
X = houseprice_df[['overallqual','grlivarea','garagecars','garagearea','totalbsmtsf','firstflrsf']]

X = sm.add_constant(X)

result = sm.OLS(Y,X).fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:              saleprice   R-squared:                       0.762
Model:                            OLS   Adj. R-squared:                  0.761
Method:                 Least Squares   F-statistic:                     775.0
Date:                Sun, 18 Aug 2019   Prob (F-statistic):               0.00
Time:                        18:26:17   Log-Likelihood:                -17496.
No. Observations:                1460   AIC:                         3.501e+04
Df Residuals:                    1453   BIC:                         3.504e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -1.027e+05   4903.994    -20.932      

In [6]:
#Q: Which features are statistically significant, and which are not?

#A: except garagearea, all the other coefficients are statisitcally significant.

In [8]:
B = houseprice_df['saleprice']
A = houseprice_df[['overallqual','grlivarea','garagecars','totalbsmtsf','firstflrsf']]

A = sm.add_constant(A)

result_2 = sm.OLS(B,A).fit()
print(result_2.summary())

                            OLS Regression Results                            
Dep. Variable:              saleprice   R-squared:                       0.762
Model:                            OLS   Adj. R-squared:                  0.761
Method:                 Least Squares   F-statistic:                     928.8
Date:                Sun, 18 Aug 2019   Prob (F-statistic):               0.00
Time:                        18:30:14   Log-Likelihood:                -17497.
No. Observations:                1460   AIC:                         3.501e+04
Df Residuals:                    1454   BIC:                         3.504e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const        -1.03e+05   4901.609    -21.006      

In [10]:
#Now, exclude the insignificant features from your model. Did anything change?
#Interpret the statistically significant coefficients by quantifying their relations with the house prices. 
#Which features have a more prominent effect on house prices?
#Do the results sound reasonable to you? If not, try to explain the potential reasons.

#All the coefficients changed very slightly but about the same.  overallqual's coefficient decreased,
#while all the other coefficients increased.  I guess after removing the insignificant feature from the model,
#the weight of each variable's impact on the target variable changed.  In other words,
#after I removed the insignificant feature from the model, the model specification changed.  
#Model specification: is the process of determining which independent variables to include and exclude 
#from a regression equation...
#All the current coefficients are considered statistically significant.
#However, change in the overallqual seems to make the biggest change on the houseprice with coefficient
#value of 2.396e+04.  Then comes garagecars as the second.  
#This sounds about right as the overall quality is what makes it possible for people to compare 
#products, and whether the house has a garagecar or not makes a huge difference in the U.S.
#People would also consider if they are getting the money's worth by comparing the house size (square foot).